In [1]:
import pandas as pd

In [26]:
data = pd.read_csv("C:\\Users\\csad\\Downloads\\train_oSwQCTC\\train.csv")
data_test = pd.read_csv("C:\\Users\\csad\\Downloads\\test_HujdGe7\\test.csv")

In [27]:
print data.shape
print data_test.shape
print data.head(5)

(550068, 12)
(233599, 11)
   User_ID Product_ID Gender   Age  Occupation City_Category  \
0  1000001  P00069042      F  0-17          10             A   
1  1000001  P00248942      F  0-17          10             A   
2  1000001  P00087842      F  0-17          10             A   
3  1000001  P00085442      F  0-17          10             A   
4  1000002  P00285442      M   55+          16             C   

  Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \
0                          2               0                   3   
1                          2               0                   1   
2                          2               0                  12   
3                          2               0                  12   
4                         4+               0                   8   

   Product_Category_2  Product_Category_3  Purchase  
0                 NaN                 NaN      8370  
1                 6.0                14.0     15200  
2                 

In [28]:
X = data.iloc[:, 0:11]
y = data.iloc[:, -1]
X_test = data_test.iloc[:, 0:11]
print X.head()

   User_ID Product_ID Gender   Age  Occupation City_Category  \
0  1000001  P00069042      F  0-17          10             A   
1  1000001  P00248942      F  0-17          10             A   
2  1000001  P00087842      F  0-17          10             A   
3  1000001  P00085442      F  0-17          10             A   
4  1000002  P00285442      M   55+          16             C   

  Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \
0                          2               0                   3   
1                          2               0                   1   
2                          2               0                  12   
3                          2               0                  12   
4                         4+               0                   8   

   Product_Category_2  Product_Category_3  
0                 NaN                 NaN  
1                 6.0                14.0  
2                 NaN                 NaN  
3                14.0         

In [15]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Product_ID', 'Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']


In [29]:
label_X_train_test = X.copy().append(X_test.copy())

# Apply one-hot encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train_test[col] = label_encoder.fit_transform(label_X_train_test[col])

label_X = label_X_train_test[0:len(X)]
label_X_test = label_X_train_test[len(X):len(label_X_train_test)]

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(label_X, y, train_size=0.8, test_size=0.2, random_state=0)

In [31]:
from sklearn.preprocessing import Imputer

imputer = Imputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))
imputed_X_test = pd.DataFrame(imputer.transform(label_X_test))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns
imputed_X_test.columms = label_X_test.columns

C:\Users\csad\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.


In [32]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=50)
rfr.fit(imputed_X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [33]:
y_preds = rfr.predict(imputed_X_valid)

In [34]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_valid, y_preds)

7492312.58399397

In [35]:
y_test_preds = rfr.predict(imputed_X_test)

In [37]:
output = pd.DataFrame({'User_ID': data_test.iloc[:, 0],
                       'Product_ID': data_test.iloc[:,1],
                       'Purchase': y_test_preds})
output.to_csv('submission_black_friday.csv', index=False)